In [2]:
from doctr.models import ocr_predictor
from doctr.io import DocumentFile

# Charger une image (remplace par ton fichier image)
doc = DocumentFile.from_images("test_facture.jpg")  # PNG, JPG, ou liste d'images

# Charger le modèle OCR
model = ocr_predictor(pretrained=True)

# Prédiction
result = model(doc)

print("result",result)


result Document(
  (pages): [Page(
    dimensions=(2339, 1653)
    (blocks): [Block(
      (lines): [
        Line(
          (words): [
            Word(value='Facture', confidence=0.97),
            Word(value='No', confidence=0.95),
            Word(value='FA42401302', confidence=1.0),
          ]
        ),
        Line(
          (words): [Word(value='eiG', confidence=0.98)]
        ),
        Line(
          (words): [
            Word(value='EIG', confidence=1.0),
            Word(value='Sud-Ouest', confidence=0.55),
          ]
        ),
        Line(
          (words): [
            Word(value='Porte', confidence=0.99),
            Word(value='de', confidence=1.0),
            Word(value='Bègles', confidence=1.0),
          ]
        ),
        Line(
          (words): [
            Word(value='Hopital', confidence=0.91),
            Word(value='privé', confidence=1.0),
            Word(value='Wallerstein', confidence=0.61),
          ]
        ),
        Line(
          (wor

In [11]:
result.pages

[Page(
   dimensions=(2339, 1653)
   (blocks): [Block(
     (lines): [
       Line(
         (words): [
           Word(value='Facture', confidence=0.97),
           Word(value='No', confidence=0.95),
           Word(value='FA42401302', confidence=1.0),
         ]
       ),
       Line(
         (words): [Word(value='eiG', confidence=0.98)]
       ),
       Line(
         (words): [
           Word(value='EIG', confidence=1.0),
           Word(value='Sud-Ouest', confidence=0.55),
         ]
       ),
       Line(
         (words): [
           Word(value='Porte', confidence=0.99),
           Word(value='de', confidence=1.0),
           Word(value='Bègles', confidence=1.0),
         ]
       ),
       Line(
         (words): [
           Word(value='Hopital', confidence=0.91),
           Word(value='privé', confidence=1.0),
           Word(value='Wallerstein', confidence=0.61),
         ]
       ),
       Line(
         (words): [
           Word(value='1', confidence=1.0),
           W

In [13]:
import json

# Export JSON complet (structure hiérarchique : blocs, lignes, mots, etc.)
exported = result.export()
with open("ocr_result.json", "w", encoding="utf-8") as f:
    json.dump(exported, f, ensure_ascii=False, indent=2)

In [16]:
#open the last json file
with open("ocr_result.json", "r", encoding="utf-8") as f:
    ocr_data = json.load(f)


[{'page_idx': 0,
  'dimensions': [2339, 1653],
  'orientation': {'value': None, 'confidence': None},
  'language': {'value': None, 'confidence': None},
  'blocks': [{'geometry': [[0.05642971585753176, 0.04296875],
     [0.9449521230338778, 0.9873046875]],
    'objectness_score': 0.752794948220253,
    'lines': [{'geometry': [[0.5276367778281912, 0.04296875],
       [0.7915680060874168, 0.0595703125]],
      'objectness_score': 0.8203156590461731,
      'words': [{'value': 'Facture',
        'confidence': 0.9736305475234985,
        'geometry': [[0.5276367778281912, 0.0439453125],
         [0.6133107890955838, 0.05859375]],
        'objectness_score': 0.8392713069915771,
        'crop_orientation': {'value': 0, 'confidence': None}},
       {'value': 'No',
        'confidence': 0.9507090449333191,
        'geometry': [[0.618838144661222, 0.04296875],
         [0.6492386002722323, 0.0595703125]],
        'objectness_score': 0.8181123733520508,
        'crop_orientation': {'value': 0, 'con

In [24]:
ocr_data['pages'][0]['blocks'][0].keys()

dict_keys(['geometry', 'objectness_score', 'lines', 'artefacts'])

In [25]:
x = ocr_data['pages'][0]['blocks'][0] 

In [50]:
geo =x['geometry']
os =x['objectness_score']
lines =x['lines']
arte =x['artefacts']

In [58]:
liste_words = []
liste_boxes = []

for i in lines:
    for j in i['words']:
        liste_words.append(j['value'])
        liste_boxes.append([j['geometry'][0][0],j['geometry'][0][1],j['geometry'][1][0],j['geometry'][1][1]])

In [97]:
import requests

def query_ollama(model, prompt):

    # 7. Appel à Ollama
    response = requests.post("http://localhost:11434/api/generate", json={
        "model": model,
        "prompt": prompt,
        "stream": False
    })

    return response.json()["response"].strip()

In [98]:
def generate_prompt_for_llm(liste_words, liste_boxes):
    # Calcul des centres des mots
    words_with_pos = []
    for word, box in zip(liste_words, liste_boxes):
        x_center = round((box[0] + box[2]) / 2, 4)
        y_center = round((box[1] + box[3]) / 2, 4)
        words_with_pos.append((word, x_center, y_center))

    # Tri des mots par Y puis X
    words_with_pos.sort(key=lambda x: (x[2], x[1]))

    # Construction du prompt
    prompt = """Voici un OCR brut d’une facture avec les positions (x, y) dans la page. Réponds uniquement avec le JSON demandé ci-dessous, sans aucune explication, texte ou commentaire. :

{
  "entreprise": "",
  "numero_facture_ou_piece": "",
  "date": "",
  "montant_TTC": "",
  "montant_TVA": "",
  "nom_client": "",
  "code_client": ""
}

Texte OCR trié (x, y normalisés) :
"""

    for word, x, y in words_with_pos:
        prompt += f"• ({x}, {y}) → \"{word}\"\n"

    return prompt

In [99]:
prompt = generate_prompt_for_llm(liste_words, liste_boxes)
response = query_ollama('mistral',prompt)

In [100]:
prompt

'Voici un OCR brut d’une facture avec les positions (x, y) dans la page. Réponds uniquement avec le JSON demandé ci-dessous, sans aucune explication, texte ou commentaire. :\n\n{\n  "entreprise": "",\n  "numero_facture_ou_piece": "",\n  "date": "",\n  "montant_TTC": "",\n  "montant_TVA": "",\n  "nom_client": "",\n  "code_client": ""\n}\n\nTexte OCR trié (x, y normalisés) :\n• (0.7239, 0.0508) → "FA42401302"\n• (0.5705, 0.0513) → "Facture"\n• (0.634, 0.0513) → "No"\n• (0.1552, 0.0825) → "eiG"\n• (0.2368, 0.1533) → "Sud-Ouest"\n• (0.1787, 0.1538) → "EIG"\n• (0.6679, 0.1655) → "Wallerstein"\n• (0.2202, 0.167) → "de"\n• (0.5504, 0.167) → "Hopital"\n• (0.6016, 0.167) → "privé"\n• (0.1863, 0.1675) → "Porte"\n• (0.2603, 0.1685) → "Bègles"\n• (0.3238, 0.1826) → "Bâtiment"\n• (0.3673, 0.1826) → "A"\n• (0.1718, 0.1831) → "1"\n• (0.1988, 0.1831) → "Quai"\n• (0.2478, 0.1831) → "Wilson"\n• (0.281, 0.1836) → "-"\n• (0.585, 0.1968) → "boulevard"\n• (0.6534, 0.1968) → "JAVAL"\n• (0.5332, 0.1973) → "14

In [101]:
response

'Here is the English translation of the text you provided:\n\n   **Header**\n   3 - Facture N°FA42401302\n   Date : 30/09/2024\n   Références : T140\n   A403301860 - Client :\n\n   **Body**\n   Code : TOTAL\n   Taux : 20,00 %\n   Montant : 750,00 €\n\n   A - Total\n   B - Acompte\n   C - Net\n   D - Payé ICI\n   E - Montant\n   F - Payer ICI\n   *******2100,00 € ******2100,00 € Montant : 100,00 €\n\n   Montant à Europlaza SAS au 125 avenue du capital, La Défense, tour EIG, 92927 Paris Cedex.\n   Contactez nous sur www.eig.fr ou contact@eig.fr\n\n   SIRET : 5829C FRO1382879625\n   APE : Intracommunautaire - TVA: 01.47.56.05.41 Fax: 01.47.56.05.99\n   Tél.: 01.47.56.05.xx\n\n   **Footer**\n   SAS : Société par Actions Simplifiée\n   T140 - Etablissement à laquelle la Facture est adressée.'

layoutlmv3

In [72]:
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from PIL import Image
import torch

# Chargement modèle
model_id = "nielsr/layoutlmv3-finetuned-funsd"
processor = LayoutLMv3Processor.from_pretrained(model_id, apply_ocr=False)
model = LayoutLMv3ForTokenClassification.from_pretrained(model_id)

# Forcer usage CPU
device = torch.device("cpu")
model = model.to(device)

# Image
image = Image.open("test_facture.jpg").convert("RGB")

# Encodage
encoding = processor(
    image,
    text=liste_words,
    boxes=liste_boxes,
    return_tensors="pt",
    truncation=True,
    padding="max_length"
)

# Cast type et move to CPU
for k in ['input_ids', 'attention_mask', 'token_type_ids', 'bbox']:
    if k in encoding and encoding[k].dtype != torch.long:
        encoding[k] = encoding[k].long()
for k in encoding:
    encoding[k] = encoding[k].to(device)

# Inference
with torch.no_grad():
    outputs = model(**encoding)
preds = torch.argmax(outputs.logits, dim=-1)

# Labels
label_map = model.config.id2label
predicted_labels = [label_map[label_id.item()] for label_id in preds[0]]

# Affichage
print(list(zip(liste_words, predicted_labels)))


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'LayoutLMv3TokenizerFast'.
c:\Users\aandre\Anaconda3\envs\doctr_cpu\lib\site-packages\transformers\modeling_utils.py:1575: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[('Facture', 'I-ANSWER'), ('No', 'I-ANSWER'), ('FA42401302', 'I-ANSWER'), ('eiG', 'I-ANSWER'), ('EIG', 'I-ANSWER'), ('Sud-Ouest', 'I-ANSWER'), ('Porte', 'I-ANSWER'), ('de', 'I-ANSWER'), ('Bègles', 'I-ANSWER'), ('Hopital', 'I-ANSWER'), ('privé', 'I-ANSWER'), ('Wallerstein', 'I-ANSWER'), ('1', 'I-ANSWER'), ('Quai', 'I-ANSWER'), ('Wilson', 'I-ANSWER'), ('-', 'I-ANSWER'), ('Bâtiment', 'I-ANSWER'), ('A', 'I-ANSWER'), ('33130', 'I-ANSWER'), ('Bègles', 'I-ANSWER'), ('14', 'I-ANSWER'), ('boulevard', 'I-ANSWER'), ('JAVAL', 'I-ANSWER'), ('Tél.', 'I-ANSWER'), ('05', 'I-ANSWER'), ('56', 'I-ANSWER'), ('75', 'I-ANSWER'), ('59', 'I-ANSWER'), ('15', 'I-ANSWER'), ('Courriel', 'I-ANSWER'), ('Facturation', 'I-ANSWER'), (':', 'I-ANSWER'), ('serviceadv@eig.fr', 'I-ANSWER'), ('33740', 'I-ANSWER'), ('ARES', 'I-ANSWER'), ('Date', 'I-ANSWER'), ('Numéro', 'I-ANSWER'), ('F', 'I-ANSWER'), ('Client', 'I-ANSWER'), ('Délai', 'I-ANSWER'), ('de', 'I-ANSWER'), ('règlement', 'I-ANSWER'), ('Mode', 'I-ANSWER'), ('de', 'I-

In [ ]:
import cv2
import numpy as np

img_color = cv2.imread("test_facture.jpg")
img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

# Détection des contours
edges = cv2.Canny(img_gray, 50, 150, apertureSize=3)

# Détection de lignes
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=200, minLineLength=100, maxLineGap=10)

# Dessin des lignes détectées sur l’image couleur
if lines is not None:
    for i, line in enumerate(lines):
        x1, y1, x2, y2 = line[0]
        cv2.line(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)  # rouge
        print(f"Ligne {i+1} : ({x1}, {y1}) -> ({x2}, {y2})")

# Sauvegarde ou affichage de l’image avec les lignes
cv2.imwrite("facture_lignes_detectees.jpg", img_color)